# Script para obtener los datos de interes


In [8]:
# cargar las bibliotecas que usaremos

import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import numpy as np
import xarray as xr
from netCDF4 import Dataset
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
from matplotlib.cm import get_cmap
from wrf import to_np, getvar, CoordPair, vertcross, interpline
import wrf
import metpy.calc as mpcalc
from metpy.plots import SkewT
from metpy.units import units

import warnings
warnings.filterwarnings('ignore')

In [9]:
meses = ['enero','febrero','marzo','abril','mayo','junio','julio','agosto','septiembre','octubre','noviembre','diciembre']
filename = []
for i,mes in zip(range(1,13),meses):#mes 
    for j in range(1,32):
        if i==1 or i==3 or i==5 or i==7 or i==8 or i==10 or i==12:
            if i<=9 and j<=9:
                filename.append('/ServerData/WRF_Pronostico2/2022/0%d_%s/wrfout_d02_2022-0%d-0%d_00.nc'%(i,mes,i,j))
            elif i<=9 and j>9:
                filename.append('/ServerData/WRF_Pronostico2/2022/0%d_%s/wrfout_d02_2022-0%d-%d_00.nc'%(i,mes,i,j))
            elif i>9 and j<=9:
                filename.append('/ServerData/WRF_Pronostico2/2022/%d_%s/wrfout_d02_2022-%d-0%d_00.nc'%(i,mes,i,j))
            elif i>9 and j>9:
                filename.append('/ServerData/WRF_Pronostico2/2022/%d_%s/wrfout_d02_2022-%d-%d_00.nc'%(i,mes,i,j))
            
                
    for j0 in range(1,29):
        if i==2:
            if j0<=9:
                filename.append('/ServerData/WRF_Pronostico2/2022/0%d_%s/wrfout_d02_2022-0%d-0%d_00.nc'%(i,mes,i,j0))
            else:
                filename.append('/ServerData/WRF_Pronostico2/2022/0%d_%s/wrfout_d02_2022-0%d-%d_00.nc'%(i,mes,i,j0))
                
    for j1 in range(1,31):
        if i==4 or i==6 or i==9 or i==11:
            if i<=9 and j1<=9:
                filename.append('/ServerData/WRF_Pronostico2/2022/0%d_%s/wrfout_d02_2022-0%d-0%d_00.nc'%(i,mes,i,j1))
            elif i<=9 and j1>9:
                filename.append('/ServerData/WRF_Pronostico2/2022/0%d_%s/wrfout_d02_2022-0%d-%d_00.nc'%(i,mes,i,j1))
            elif i>9 and j1<=9:
                filename.append('/ServerData/WRF_Pronostico2/2022/%d_%s/wrfout_d02_2022-%d-0%d_00.nc'%(i,mes,i,j1))
            elif i>9 and j1>9:
                filename.append('/ServerData/WRF_Pronostico2/2022/%d_%s/wrfout_d02_2022-%d-%d_00.nc'%(i,mes,i,j1))
            

In [16]:
lat1 = 19.06
lat2 = 19.6
lon1 = -99.5
lon2 = -99.23


def calc_curvature(U,z):
    '''Calculate second derivative of U wrt z'''
    d2Udz2 = (U[2:,...] - 2*U[1:-1,...] + U[0:-2,...]) / ((z[2:,...]-z[1:-1,...])*(z[1:-1,...]-z[0:-2,...]))
    return d2Udz2

#for nombre in filename[:5]:
nombre='wrfout_d02_2022-01-11_00.nc'
ncfile=Dataset(nombre)
# cargar DataFrame con xarray
ds = xr.open_dataset(nombre)
print(nombre)
x_y_1 = wrf.ll_to_xy(ncfile, lat1, lon1).data
x_y_2 = wrf.ll_to_xy(ncfile, lat2, lon2).data

ilat_1 = x_y_1[1]
ilat_2 = x_y_2[1]
ilon_1 = x_y_1[0]
ilon_2 = x_y_2[0]

altm_aju=[]
latm_aju=[]
lonm_aju=[]
ilatm_aju=[]
ilonm_aju=[]
t=20
for i in range(ilat_1,ilat_2):#lat
    for j in range(ilon_1,ilon_2):#lon
        if ds.HGT[t,i,j]>3200:
            altm_aju.append(ds.HGT[t,i,j])
            latm_aju.append(ds.XLAT[t,i,j])
            lonm_aju.append(ds.XLONG[t,i,j])
            ilatm_aju.append(i)
            ilonm_aju.append(j)

altm_aju = np.array(altm_aju)
latm_aju = np.array(latm_aju)
lonm_aju = np.array(lonm_aju)
ilatm_aju = np.array(ilatm_aju)
ilonm_aju = np.array(ilonm_aju)


u_wrf=np.empty((25,49,156,273))
v_wrf=np.empty((25,49,156,273))
th_wrf=np.empty((25,49,156,273))
for i in range(25):
    u_wrf[i,:,:,:]=getvar(ncfile,'ua', timeidx=i, units='m s-1')
    v_wrf[i,:,:,:]=getvar(ncfile,'va', timeidx=i, units='m s-1')
    th_wrf[i,:,:,:]=getvar(ncfile,'th', timeidx=i, units='K')

n_lat_aju=len(latm_aju)
theta = np.deg2rad(25)
#Recordemos rotar 
z=getvar(ncfile,'z',units='m')
u_wrf_aju=np.empty((25,49,n_lat_aju))
th_wrf_aju=np.empty((25,49,n_lat_aju))
z_wrf_aju=np.empty((49,n_lat_aju))
for t in range(25):
    for k in range(49):
        for i in range(n_lat_aju):
            #print(ilatm_aju[i],ilonm_aju[i])
            c1=np.cos(theta)*u_wrf[t,k,ilatm_aju[i],ilonm_aju[i]]+np.sin(theta)*v_wrf[t,k,ilatm_aju[i],ilonm_aju[i]]
            u_wrf_aju[t,k,i] = c1
            

for i in range(n_lat_aju):
    z_wrf_aju[:,i]=z[:,ilatm_aju[i],ilonm_aju[i]]
    th_wrf_aju[:,:,i]=th_wrf[:,:,ilatm_aju[i],ilonm_aju[i]]

u_wrf_aju = u_wrf_aju*units('m/s')
z_wrf_aju = z_wrf_aju*units.meters
th_wrf_aju = th_wrf_aju*units.kelvin

#CALCULO DE LA REGLA DEL 1.6
#Cómo ya sabemos que en su mayoria en el nivel 23 tenemos una 
#diferencia de 2000 m, ahora calculamos si hay una diferencia 
#de 1.6 veces entre la  velocidad en la superficie de la montaña
#y 2000 metros más arriba
hrs=[]
pts=[]
for t in range(25):
    for i in range(n_lat_aju):
        if u_wrf_aju[t,25,i]>=u_wrf_aju[t,0,i]*1.6:
            #print("se cumple a la hora %f GMT el dato %d"%(t,i))
            hrs.append(t)
            pts.append(i)

l2_wrf = np.empty((25,47,12))
N2_wrf = np.empty((25,49,12))
for i in range(0,25,1):

    N2=mpcalc.brunt_vaisala_frequency_squared(z_wrf_aju,th_wrf_aju[i,:,:])
    N2_wrf[i,:,:]=N2
    d2Udz2_wrf=calc_curvature(u_wrf_aju[i,:,:],z_wrf_aju)
    l2_wrf[i,:,:] = (N2[1:-1]/u_wrf_aju[i,1:-1,:]**2)-d2Udz2_wrf/u_wrf_aju[i,1:-1,:]
    #nt(l2_wrf)
narch=('datos_%s.npz'%nombre[-16:])
np.savez(narch,lat_aju=latm_aju,lon_aju=lonm_aju,ilatm_aju=ilatm_aju,
         ilonm_aju=ilonm_aju,z_aju=z_wrf_aju.magnitude,hrs_16=hrs,pts16=pts,
         l2=l2_wrf,N2=N2_wrf,u_aju=u_wrf_aju.magnitude,)

datos=np.load(narch)
print(narch)
print(datos.files)

wrfout_d02_2022-01-11_00.nc
datos_2022-01-11_00.nc.npz
['lat_aju', 'lon_aju', 'ilatm_aju', 'ilonm_aju', 'z_aju', 'hrs_16', 'pts16', 'l2', 'N2', 'u_aju']


In [19]:
l2_wrf[7,20,0]

1.726428491931641e-05